In [31]:
# import libraries
import os
import re
import json
import sys
import cv2
import time
import string
import selenium
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [40]:
# load chrome driver and the site
driver_path = "/home/ashwinkumar/Projects/Driver/chromedriver_linux64/chromedriver"
default_location = "/home/ashwinkumar/Upwork_Projects/GabrielNY"
options = webdriver.chrome.options.Options()
options.add_argument('--headless')
wd = webdriver.Chrome(executable_path=driver_path, options=options)
wd.maximize_window()
time.sleep(5)

# home url
wd.get("https://www.gabrielny.ca/")

# click ok on cookies
if wd.find_element_by_css_selector("div.cookies-button button[type='button']").is_displayed():
    wd.find_element_by_css_selector("div.cookies-button button[type='button']").click()
    
# click the item page drop down and click the 80 per page button
# wd.find_element_by_css_selector("div.row a.dropdown-select").click()
# time.sleep(1)
# [i for i in wd.find_elements_by_css_selector("ul#sub-display2 li a.modepp") if "80" in i.text][0].click()

In [41]:
# list of metals from the website
metals_list = ["white-gold", "rose-gold", "platinum", "yellow-gold", "metal"]

In [42]:
# get list of all categories
categories = wd.find_elements_by_css_selector("ul.menu.thikbr.hidden-sm-up li.sub a")
category_links = [category.get_attribute("href") for category in categories if any([re.findall(i, category.get_attribute("href")) for i in metals_list])]

In [43]:
category_links[:2]

['https://www.gabrielny.ca/engagement-rings/platinum-engagement-rings',
 'https://www.gabrielny.ca/engagement-rings/white-gold-engagement-rings']

In [44]:
# fetch the window height and scroll to the end to load more images
def scroll_to_height(wd, height):
    wd.execute_script("window.scrollTo(0, {})".format(height))

# function to return list of all products and links in the page
def get_product_links_and_names(wd, link, limit):
    
    wd.get(link)
    products = wd.find_elements_by_css_selector("div.product.newprinfo")
    load_more_button = wd.find_element_by_css_selector("a.listingloadmore.btn.btn1")
    counter = 1

    while True:
        if counter % 10 == 0:
            print(len(products))

        if load_more_button.is_displayed():
            load_more_button.click()

        try:
            new_location = products[-1].location["y"] + 100
            scroll_to_height(wd, new_location)
            time.sleep(10)
            new_products = wd.find_elements_by_css_selector("div.product.newprinfo")

            # check the number of products
            print("Counter:", counter)
            print(len(set(products)), len(set(new_products)))
            print("-----")

#             if len(set(new_products)) <= len(set(products)):
            if counter > limit:
                break
            products.extend(new_products)
            counter += 1

        except Exception as e:
            print(e)
            break

    products = list(set(products))
    # get individual product names and their urls
    product_names = [i.find_element_by_css_selector("div.product-info a").text for i in products]
    product_links = [i.find_element_by_css_selector("div.product-info a").get_attribute("href") for i in products]

    return product_names, product_links

In [45]:
# function to save images to the desired folder
def save_images_to_folder(image_urls, folder_location, folder_name, extension):
    
    folder_path = os.path.join(folder_location, folder_name)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    for i in range(len(image_urls)):
        
        file_name = "{}_{}.{}".format(folder_name, i + 1, extension)
        file_name = os.path.join(folder_path, file_name)
        image = requests.get(image_urls[i]).content
        image = Image.open(BytesIO(image)).convert("RGB")
        image.save(file_name)

In [46]:
# function to get the product name, direct to the link and then get the price and product details
def get_product_details(wd, product_links, product_names):
    
    product_list = []
    counter = 0
    for link in product_links:
        wd.get(link)
#         time.sleep(4)
        prices = wd.find_elements_by_css_selector("div.box_span.pb3 span.price")
        price = [p.text for p in prices if p.text != ""][0]
        table = wd.find_elements_by_css_selector("div.dtl-right-inner table.mt15 tr")
        details = [row.text.split(":") for row in table if len(row.text.split(":")) > 1]
        details = [(re.sub(r"\n", "", k), re.sub(r"\n", "", v)) for k, v in details]
        details.extend([["Product_Name", product_names[counter]], ["Price", price]])
        image_urls = [i.get_attribute("src") for i in wd.find_elements_by_css_selector("div.owl-item img.prod-zoom-img")]
        details_dict = dict(details)
        product_list.append(details_dict)
        # save images to the folder
        save_images_to_folder(image_urls, default_location, details_dict["Item Code"], extension="jpg")
        print("{} product fetched successfully".format(product_names[counter]))
        counter += 1

        if counter % 100 == 0:
            print(counter)
    return product_list

In [47]:
# main function to collect all product descriptions
for category_link in category_links[:1]:
    
    product_names, product_links = get_product_links_and_names(wd, category_link, limit=0)
    print("---------{} category fetched successfully".format(category_link.split("/")[-1]))
    new = get_product_details(wd, product_links, product_names)

Counter: 1
16 32
-----
---------platinum-engagement-rings category fetched successfully
Dane - Platinum Round Diamond Engagement Ring product fetched successfully
Lunaria - Vintage Inspired Platinum Round Three Stone Sapphire and Diamond Engagement Ring product fetched successfully
Kayla - Platinum Round Twisted Diamond Engagement Ring product fetched successfully
Joanna - Platinum Round Diamond Engagement Ring product fetched successfully
Oyin - Platinum Round Diamond Engagement Ring product fetched successfully
Megan - Platinum Round Diamond Engagement Ring product fetched successfully
Stacy - Platinum Round Halo Diamond Engagement Ring product fetched successfully
Lana - Platinum Round Halo Diamond Engagement Ring product fetched successfully
Cruz - Vintage Inspired Platinum Round Three Stone Sapphire and Diamond Engagement Ring product fetched successfully
Morgan - Platinum Round Twisted Diamond Engagement Ring product fetched successfully
Elyse - Platinum Round Split Shank Diamond

In [48]:
new

[{'Product Line': 'Dane',
  'Item Code': 'ER7389PT3JJ',
  'Crown Type': 'Not changeable',
  'Center Stone Carat Weight': '1.00 ct',
  'Center Stone Size': '6.5 mm',
  'Total Diamond Weight (TW)': '0.14 ct',
  'Division': 'Amavida',
  'Collection': 'Victorian',
  'Style': 'Straight',
  'Stone Shape': 'Round',
  'Product_Name': 'Dane - Platinum Round Diamond Engagement Ring',
  'Price': 'CAD 2,205'},
 {'Product Line': 'Lunaria',
  'Item Code': 'ER10042PT3SA',
  'Crown Type': 'Not changeable',
  'Center Stone Carat Weight': '1 ct',
  'Center Stone Size': '6.5 mm',
  'Total Diamond Weight (TW)': '0.17 ct',
  'Division': 'Amavida',
  'Collection': 'Victorian',
  'Style': '3 Stone',
  'Diamond Color': 'G-H Color',
  'Diamond Clarity': 'VS2 Clarity',
  'Stone Shape': 'Round',
  'Product_Name': 'Lunaria - Vintage Inspired Platinum Round Three Stone Sapphire and Diamond Engagement Ring',
  'Price': 'CAD 2,930'},
 {'Product Line': 'Kayla',
  'Item Code': 'ER7805PT4JJ',
  'Crown Type': 'Changeabl

In [49]:
# create dataframe from scraped data
pd.DataFrame.from_dict(new)

,Product Line,Item Code,Crown Type,Center Stone Carat Weight,Center Stone Size,Total Diamond Weight (TW),Division,Collection,Style,Stone Shape,Product_Name,Price,Diamond Color,Diamond Clarity
0,Dane,ER7389PT3JJ,Not changeable,1.00 ct,6.5 mm,0.14 ct,Amavida,Victorian,Straight,Round,Dane - Platinum Round Diamond Engagement Ring,"CAD 2,205",NaN,NaN
1,Lunaria,ER10042PT3SA,Not changeable,1 ct,6.5 mm,0.17 ct,Amavida,Victorian,3 Stone,Round,Lunaria - Vintage Inspired Platinum Round Thre...,"CAD 2,930",G-H Color,VS2 Clarity
2,Kayla,ER7805PT4JJ,Changeable ( 0.50ct - 2.00ct ),1.00 ct,6.5 mm,0.35 ct,Gabriel Bridal,Contemporary,Twisted,Round,Kayla - Platinum Round Twisted Diamond Engagem...,"CAD 3,685",NaN,NaN
3,Joanna,ER7224PT4JJ,Changeable ( 0.50ct - 2.00ct ),1 ct,6.5 mm,0.23 ct,Gabriel Bridal,Classic,Straight,Round,Joanna - Platinum Round Diamond Engagement Ring,"CAD 2,700",NaN,NaN
4,Oyin,ER4181PT4JJ,Changeable ( 0.50ct - 3.00ct ),1 ct,6.5 mm,0.16 ct,Gabriel Bridal,Classic,Straight,Round,Oyin - Platinum Round Diamond Engagement Ring,"CAD 2,290",NaN,NaN
5,Megan,ER7537PT4JJ,Not changeable,1 ct,6.5 mm,0.23 ct,Gabriel Bridal,Classic,Straight,Round,Megan - Platinum Round Diamond Engagement Ring,"CAD 2,630",NaN,NaN
6,Stacy,ER7265PT4JJ,Not changeable,1 ct,6.5 mm,0.26 ct,Gabriel Bridal,Contemporary,Halo,Round,Stacy - Platinum Round Halo Diamond Engagement...,"CAD 3,310",NaN,NaN
7,Lana,ER7495PT4JJ,Not changeable,1 ct,6.5 mm,0.39 ct,Gabriel Bridal,Classic,Halo,Round,Lana - Platinum Round Halo Diamond Engagement ...,"CAD 3,685",NaN,NaN
8,Cruz,ER8773PT3SA,Not changeable,1 ct,6.5 mm,0.19 ct,Amavida,Victorian,3 Stone,Round,Cruz - Vintage Inspired Platinum Round Three S...,"CAD 3,340",G-H Color,VS2 Clarity
9,Morgan,ER10439PT4JJ,Changeable ( 0.50ct - 2.00ct ),1 ct,6.5 mm,0.18 ct,Gabriel Bridal,Contemporary,Twisted,Round,Morgan - Platinum Round Twisted Diamond Engage...,"CAD 3,845",NaN,NaN


In [50]:
# disconnect the webdriver
wd.quit()